# Routing Chains
Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user input. In these types of chains, there is a "router" LLM chain which has access to a suite of tools. Depending on the user input, the router can then decide which, if any, of these tools to call.

These types of chains are called Routing Chains. When used correctly these can be extremely powerful. The purpose of this notebook is to go through the core abstractions at the heart of Routing Chains and enable you to build applications with them.

## Terminology

Before going through any code, let's align on some terminology.
- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
- Tool Input: The input string to a tool.
- Observation: The output from calling a tool on a particular input.
- Router: The object responsible for deciding which tools to call and when. Exposes a `route` method, which takes in a string and returns a Router Output.
- Router Output: The object returned from calling `Router.route` on a string. Consists of:
    - The tool to use
    - The input to that tool
    - A log of the router's thinking. This is not required, as it's only used for logging purposes, but it can be helpful for understanding why the router is acting why it is
- Routing Chain: A chain which is made up of a router and suite of tools. When passed a string, the Routing Chain will iterative call tools as needed until it arrives at a Final Answer.
- Final Answer: The final output of a Routing Chain.

## Tools
When constructing your own Routing Chain, you will need to provide it with a list of tools that it can use. This is done with a list of ToolConfigs. The ToolConfig is used not only to create the Routing Chain, but is also sometimes used to create the router itself (often, the router logic depends on the tools available). 

```python
class ToolConfig(NamedTuple):
    """Configuration for tools."""

    tool_name: str
    tool: Callable[[str], str]
    # Needed to construct some routers.
    tool_description: Optional[str] = None
```

The two required components of a ToolConfig are the name and then the tool itself. A tool description is optional, as it is needed for some routers but not all.

## Router
A central piece of this chain is the router. The router is responsible for taking user input and deciding which tools, if any, to use. Although it doesn't necessarily have to be backed by a language model (LLM), for pretty much all current use cases it is. LLMs make great routers because they are really good at understanding human intent, which makes them perfect for choosing which tools to use (and for interpreting the output of those tools).

Below is the interface we expect routers to expose, along with the RouterOutput definition.

```python

class RouterOutput(NamedTuple):
    """Output of a router."""

    tool: str
    tool_input: str
    log: str
        

class Router(ABC):
    """Chain responsible for deciding the action to take."""

    @abstractmethod
    def route(self, text: str) -> RouterOutput:
        """Given input, decided how to route it.

        Args:
            text: input string

        Returns:
            RouterOutput specifying what tool to use.
        """

    @property
    @abstractmethod
    def observation_prefix(self) -> str:
        """Prefix to append the observation with before calling the router again."""

    @property
    @abstractmethod
    def router_prefix(self) -> str:
        """Prefix to prepend the router call with."""

    @property
    def finish_tool_name(self) -> str:
        """Name of the tool to use to finish the chain."""
        return "Final Answer"

    @property
    def starter_string(self) -> str:
        """Put this string after user input but before first router call."""
        return "\n"
```

## Routing Chains
The above are definitions for Routers and ToolConfigs. These are useful if you want to create your own Router. In practice, you will most likely use an existing RoutingChain. Below we cover a few different types of RoutingChains. Please select the one that best suites your needs.

### ReAct on Docstore
To construct this RoutingChain, please provide a `Docstore` object along with the LLM to use as the router:

```python
chain = ReActChain(llm, docstore)
```

### Self-Ask with Search
To construct this RoutingChain, please provide a `Search` object along with the LLM to use as the router.

```python
chain = SelfAskWithSearch(llm, search)
```

### MRKL Zero Shot
To construct this RoutingChain, a `tool_description` must be specified for each tool. You may provide any number of tools, along side the LLM to use as a router.

```python
chain = MRKLChain.from_tools(llm, tools)
```